In [ ]:
import os
import cv2 as cv
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import keras
from tensorflow.keras.models import Model,  Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout, BatchNormalization, Activation
from sklearn.metrics import confusion_matrix , classification_report, accuracy_score, recall_score, precision_score, f1_score

In [ ]:
# generating dataset code named gen_seg.ipynb
# dividing dataset named split.py

In [ ]:
# path to the datasets

# Base directory where your datasets are located
base_dir = r'E:\Desktop\Carselona\Gate open or close\dataset'

train_path = os.path.join(base_dir, 'train')
val_path = os.path.join(base_dir, 'val')
# test_path = os.path.join(base_dir, 'test')


In [ ]:
# showing the images of each class
for folder in os.listdir(train_path):
    sub_path = train_path + "/" + folder
    print(folder)
    for i in range(2):
        temp_path = os.listdir(sub_path)[i]
        temp_path = sub_path + "/" + temp_path
        img = mpimg.imread(temp_path)
        implot = plt.imshow(img)
        plt.show()

In [ ]:
# convert image into  pixels here feature is edge
def imagearray(path, size):
    data = []
    for folder in os.listdir(path):
        sub_path = os.path.join(path, folder)
        for img in os.listdir(sub_path):
            image_path = os.path.join(sub_path, img)
            img_arr = cv.imread(image_path)

            # Resize the image
            img_arr = cv.resize(img_arr, size)

            # Apply Canny on a copy of a channel (e.g., grayscale)
            edge_mask = cv.Canny(img_arr[:, :, 0], 40, 200)

            # Create a 3-channel image by stacking the edge mask along the third axis
            edge_img = np.stack([edge_mask] * 3, axis=-1)

            data.append(edge_img)
    return data

In [ ]:
# size is 250 * 250 for VGG
size = (250,250)

In [ ]:
# apply above convert function on train test val 
train = imagearray(train_path,size)
val = imagearray(val_path,size)
# test = imagearray(test_path,size)

In [ ]:
# Normalisation
x_train = np.array(train)
x_val   = np.array(val)
# x_test  = np.array(test)

x_train = x_train/255
x_val   = x_val/255
# x_test  = x_test/255

In [ ]:
# checking the input shape .... must be 4 channel (n , 250,250, 3) where n is number of images
x_train.shape

In [ ]:
# defining target variables
def data_class(data_path, size, class_mode):
    datagen = ImageDataGenerator(rescale=1./255)
    classes = datagen.flow_from_directory(data_path, target_size= size, batch_size = 32, class_mode = class_mode)
    return classes

train_class = data_class(train_path, size,"sparse")
val_class   = data_class(val_path, size, "sparse")
# test_class  = data_class(test_path, size, "sparse")


In [ ]:
y_train = train_class.classes
y_val = val_class.classes
# y_test = test_class.classes

In [ ]:
# conversion to class index
train_class.class_indices

In [ ]:
# check shape
print(y_train.shape,y_val.shape)

In [ ]:
# initialise VGG
vgg = VGG19(input_shape = (250,250,3) , weights = "imagenet" , include_top = False)
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(2, activation = "softmax")(x)

model = Model(inputs = vgg.input, outputs = prediction)
model.summary()

early_stop = EarlyStopping(monitor = "val_loss" , mode= "min", verbose=1, patience = 5)

In [ ]:
# compile the model architecture
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

In [ ]:
# fit the model ..set epochs 50+ 
history = model.fit(x_train,y_train,
          validation_data = (x_val,y_val),
          epochs = 15,
          callbacks = [early_stop],
          batch_size = 30,
          shuffle = True)

In [ ]:
# Assuming 'model' is your trained Keras model save it
model.save('test_door.h5')

In [ ]:
# Plotting accuracy
plt.figure(figsize=(10, 8))
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.title("Accuracy")

# Specify the path and filename to save the plot
save_path = 'metrices\accuracy_plot.png' 

# Save the plot to the specified file
plt.savefig(save_path)

# Must be increasing function

In [ ]:
# Plotting loss
plt.figure(figsize=(10, 8))
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.title('Loss')

# Specify the path and filename to save the plot
save_path_loss = 'metrices\loss_plot.png'  # Change this to your desired filename and path for loss plot

# Save the loss plot to the specified file
plt.savefig(save_path_loss)

# must be decreasing function

In [ ]:
# Evaluate model
# before doing this uncomment the test folder everywhere
model.evaluate(x_test, y_test, batch_size = 32)

In [ ]:
# prediction for metrices
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
# Classification report
print(classification_report(y_pred,y_test))

In [ ]:
# F1 score
print('F1 score:\t')
f1_score(y_test,y_pred, average = 'micro')

# recall_score
print('recall_score: \t')

In [ ]:
# Precision score
print('precision_score:\t')
precision_score(y_test, y_pred, average='micro')

In [ ]:
# confusion metric 
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Display the confusion matrix using seaborn heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={"size": 16})

# Add labels, title, and ticks
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.xticks([0.5, 1.5], ['Predicted 0', 'Predicted 1'])
plt.yticks([0.5, 1.5], ['Actual 0', 'Actual 1'])

# Specify the path and filename to save the plot
save_path_conf = 'metrices\confusion_metric.png' 

# Save the loss plot to the specified file
plt.savefig(save_path_conf)
